from website: https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

In [2]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import os
print('ok')

ok


## Step 1: Exploring the Video Classification Dataset, we only do the classification in first 10 classess

In [3]:
mainpath = '/Users/zhennongchen/Documents/Zhennong_Video Classification/Data/UCF101'
txtpath = os.path.join(mainpath,'UCF_TrainTestlist')
videopath = os.path.join(mainpath,'UCF_Video_sub_classes1')

f = open(os.path.join(txtpath,"trainlist01.txt"),'r')
temp = f.read()
video_list = temp.split('\n')
sub_video_list = []
for v in video_list[:-1]:
    file = v.split(' ')[0]
    if os.path.isfile(os.path.join(videopath,file)) == True:
        sub_video_list.append(file)
train = pd.DataFrame()
train['video_name'] = sub_video_list
f.close()

    
f = open(os.path.join(txtpath,"testlist01.txt"),'r')
temp = f.read()
video_list = temp.split('\n')
sub_video_list = []
for v in video_list[:-1]:
    file = v.split(' ')[0]
    if os.path.isfile(os.path.join(videopath,file)) == True:
        sub_video_list.append(file)
test = pd.DataFrame()
test['video_name'] = sub_video_list
f.close()   

In [4]:
# create tags for training videos and test videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
train['tag'] = train_video_tag

test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
test['tag'] = test_video_tag
test.head()

,video_name,tag
0,Basketball/v_Basketball_g01_c01.avi,Basketball
1,Basketball/v_Basketball_g01_c02.avi,Basketball
2,Basketball/v_Basketball_g01_c03.avi,Basketball
3,Basketball/v_Basketball_g01_c04.avi,Basketball
4,Basketball/v_Basketball_g01_c05.avi,Basketball


In [5]:
# extract time frames from training and test videos
for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture(os.path.join(videopath,videoFile.split(' ')[0].split('/')[0],videoFile.split(' ')[0].split('/')[1]))
    frameRate = cap.get(5)
    
    x = 1
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret,frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename = os.path.join(mainpath,'train_1/'+videoFile.split('/')[1].split(' ')[0]+"_frame%d.jpg"%count)
            cv2.imwrite(filename,frame)
            count += 1
    cap.release()
print('done training set')

for i in tqdm(range(test.shape[0])):
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(os.path.join(videopath,videoFile.split(' ')[0].split('/')[0],videoFile.split(' ')[0].split('/')[1]))
    frameRate = cap.get(5)
    
    x = 1
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret,frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename = os.path.join(mainpath,'test_1/'+videoFile.split('/')[1].split(' ')[0]+"_frame%d.jpg"%count)
            cv2.imwrite(filename,frame)
            count += 1
    cap.release()
print('done test set')

  4%|▍         | 3/72 [00:00<00:03, 19.08it/s]

done training set


100%|██████████| 72/72 [00:02<00:00, 30.64it/s]

done test set


In [6]:
# save the name of these frames with tag in a .csv file.
images = glob(os.path.join(mainpath,'train_1','*.jpg'))
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    train_image.append(images[i].split('/')[-1])
    train_class.append(images[i].split('/')[-1].split('_')[1])

# store in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# convert the dataframe into csv file
train_data.to_csv(os.path.join(mainpath,'train_new.csv'),header=True, index=False)

100%|██████████| 907/907 [00:00<00:00, 434174.13it/s]


## Step 2: training the video classification model

In [4]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [5]:
train = pd.read_csv(os.path.join(mainpath,'train_new.csv'))
# in order to make smaller size of data, we only pick the first 30000
print(train.shape)
train.head()

(907, 2)


,image,class
0,v_Basketball_g23_c05.avi_frame2.jpg,Basketball
1,v_Basketball_g10_c03.avi_frame3.jpg,Basketball
2,v_Basketball_g15_c07.avi_frame0.jpg,Basketball
3,v_Basketball_g20_c04.avi_frame2.jpg,Basketball
4,v_Basketball_g16_c06.avi_frame0.jpg,Basketball


In [6]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(os.path.join(mainpath,'train_1',train['image'][i]),target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255 # normalize the pixel value
    train_image.append(img)

100%|██████████| 907/907 [00:02<00:00, 373.37it/s]


In [7]:
X = np.asarray(train_image)
# preprocessing the input for VGG
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X,mode = 'tf')

In [8]:
# create training and validation set
y = train['class']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2,stratify = y)
# here , striatify = y keeps the similar distribution of classess in both the training as well as the validation set.
print(X_train.shape,X_test.shape)

(725, 224, 224, 3) (182, 224, 224, 3)


In [9]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [10]:
# define the architescture of the video classification model
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
X_train = base_model.predict(X_train)
print(X_train.shape)

Instructions for updating:
Colocations handled automatically by placer.
(725, 7, 7, 512)


In [12]:
X_test = base_model.predict(X_test)
print(X_test.shape)

(182, 7, 7, 512)


In [15]:
X_train = X_train.reshape(X_train.shape[0],7*7*512)
X_test = X_test.reshape(X_test.shape[0],7*7*512)
max_val = X_train.max()
X_train = X_train/max_val
X_test = X_test/max_val
print(X_train.shape,X_test.shape)

(725, 25088) (182, 25088)


In [37]:
# build FC model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='relu'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout, so there is sync on training and validation
model.add(Dense(units=512, activation='relu'))    # hidden layer, increase number of layers to increase accuracy on training
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='relu'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=128, activation='relu'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=64, activation='relu'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(2, activation='softmax'))            # output layer

In [38]:
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(train['class']), train['class'])  # computing weights of different classes

In [40]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint(os.path.join(mainpath,'best_weight.hdf5'), save_best_only=True, verbose=1, monitor='val_loss', mode='min')

# compile the model
model.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics=['accuracy'])

# training the model
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),class_weight=class_weights,callbacks=[mcp_save],batch_size=145)

Train on 725 samples, validate on 182 samples
Epoch 1/100
725/725 [==============================] - 8s 11ms/step - loss: 0.7336 - acc: 0.5159 - val_loss: 0.6836 - val_acc: 0.6099
Epoch 2/100
725/725 [==============================] - 3s 5ms/step - loss: 0.6985 - acc: 0.5517 - val_loss: 0.6865 - val_acc: 0.6099
Epoch 3/100
725/725 [==============================] - 3s 5ms/step - loss: 0.6885 - acc: 0.5710 - val_loss: 0.6831 - val_acc: 0.6099
Epoch 4/100
725/725 [==============================] - 3s 5ms/step - loss: 0.7014 - acc: 0.5545 - val_loss: 0.6840 - val_acc: 0.6099
Epoch 5/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6784 - acc: 0.5931 - val_loss: 0.6833 - val_acc: 0.6099
Epoch 6/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6713 - acc: 0.6097 - val_loss: 0.6796 - val_acc: 0.6099
Epoch 7/100
725/725 [==============================] - 3s 5ms/step - loss: 0.6793 - acc: 0.6041 - val_loss: 0.6776 - val_acc: 0.6099
Epoch 8/100
725/725 [=

Epoch 62/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6662 - acc: 0.6124 - val_loss: 0.6689 - val_acc: 0.6099
Epoch 63/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6687 - acc: 0.6124 - val_loss: 0.6689 - val_acc: 0.6099
Epoch 64/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6663 - acc: 0.6124 - val_loss: 0.6688 - val_acc: 0.6099
Epoch 65/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6690 - acc: 0.6124 - val_loss: 0.6688 - val_acc: 0.6099
Epoch 66/100
725/725 [==============================] - 4s 6ms/step - loss: 0.6679 - acc: 0.6124 - val_loss: 0.6689 - val_acc: 0.6099
Epoch 67/100
725/725 [==============================] - 4s 5ms/step - loss: 0.6680 - acc: 0.6124 - val_loss: 0.6688 - val_acc: 0.6099
Epoch 68/100
725/725 [==============================] - 5s 7ms/step - loss: 0.6695 - acc: 0.6124 - val_loss: 0.6688 - val_acc: 0.6099
Epoch 69/100
725/725 [==============================] - 5s 6ms

## Evaluate our video classification model

In [41]:
base_model = VGG16(weights='imagenet',include_top=False)
# build FC model
m = Sequential()
m.add(InputLayer((7*7*512,)))    # input layer
m.add(Dense(units=1024, activation='relu'))   # hidden layer
m.add(Dropout(0.5))      # adding dropout, so there is sync on training and validation
m.add(Dense(units=512, activation='relu'))    # hidden layer, increase number of layers to increase accuracy on training
m.add(Dropout(0.5))      # adding dropout
m.add(Dense(units=256, activation='relu'))    # hidden layer
m.add(Dropout(0.5))      # adding dropout
m.add(Dense(units=128, activation='relu'))    # hidden layer
m.add(Dropout(0.5))      # adding dropout
m.add(Dense(units=64, activation='relu'))    # hidden layer
m.add(Dropout(0.5))      # adding dropout
m.add(Dense(2, activation='softmax'))            # output layer

In [42]:
m.load_weights(os.path.join(mainpath,'best_weight.hdf5'))

In [43]:
# compiling the model
m.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [44]:
images = glob(os.path.join(mainpath,'test_1','*.jpg'))
test_image = []
test_class = []
for i in tqdm(range(len(images))):
    test_image.append(images[i].split('/')[-1])
    test_class.append(images[i].split('/')[-1].split('_')[1])

# store in a dataframe
test_data = pd.DataFrame()
test_data['image'] = test_image
test_data['class'] = test_class

# convert the dataframe into csv file
test_data.to_csv(os.path.join(mainpath,'test_new.csv'),header=True, index=False)

100%|██████████| 263/263 [00:00<00:00, 234005.51it/s]


In [48]:
test = pd.read_csv(os.path.join(mainpath,'test_new.csv'))
print(test.shape)
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img(os.path.join(mainpath,'test_1',test['image'][i]),target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255 # normalize the pixel value
    test_image.append(img)
predict_X = np.asarray(test_image)
print(predict_X.shape)

 22%|██▏       | 57/263 [00:00<00:00, 561.29it/s]

(263, 2)


100%|██████████| 263/263 [00:00<00:00, 548.40it/s]


(263, 224, 224, 3)


In [50]:
from keras.applications.vgg16 import preprocess_input
predict_X = preprocess_input(predict_X,mode = 'tf')
predict_y = pd.get_dummies(test['class'])

In [52]:
predict_X = base_model.predict(predict_X)
print(predict_X.shape)

(263, 7, 7, 512)


In [53]:
predict_X = predict_X.reshape(predict_X.shape[0],7*7*512)
max_val = predict_X.max()
predict_X = predict_X / max_val

In [54]:
scores = m.evaluate(predict_X,predict_y)
print('acc: %.2f%%' %(scores[1]*100))

263/263 [==============================] - 1s 4ms/step
acc: 57.03%
